### Mapping Changes to the Unemployment Rate using Plotly

*Brian Dew, Domestic Program Intern, CEPR*<br>
*June 17, 2017*

Updated February 6, 2018

In [1]:
# Import libraries and sign in to plotly
import pandas as pd
import config   # File that contains api key
import plotly.plotly as py
from plotly.graph_objs import *
py.sign_in('bdew', config.plotly_key)

In [2]:
# CSV file contains city names, their BLS codes (Metropolitan 
# MTAs/NECTAs only), and their latitude and longitude
MSA = pd.read_csv('MSA.csv')

# BLS flat file contains the unemployment rate by month since 2015
url = 'https://download.bls.gov/pub/time.series/la/la.data.0.CurrentU15-19'
df = pd.read_table(url, sep='\s+', na_values=['-'])

In [3]:
# Take only the unemployment data for MSAs and monthly series 
# (M13 is annual average)
df = pd.DataFrame(df[df['series_id'].isin(MSA['fullcode']) 
                 & ~df['period'].isin(['M13'])])

# Clean up the date to convert it to pandas datetime
df.loc[:,'month'] = df['period'].replace(regex=True, 
                                         to_replace=r'M', 
                                         value=r'')
df.loc[:,'day'] = 1
df.loc[:,'date'] = pd.to_datetime(df[['year', 'month', 'day']])
df.loc[:,'value'] = df['value'].astype('float')
df = df.set_index(['series_id','date'])

In [4]:
df.loc[:,'value_ch'] = df['value'].diff(periods=24)
df = df.reset_index().groupby(['series_id']).tail(1)
df['prevvalue'] = df['value'] - df['value_ch']
#df.sort_values('value_ch')

In [5]:
df['prevvalue'] = df['value'] - df['value_ch']

In [6]:
df.sort_values('value_ch')

,series_id,date,year,period,value,footnote_codes,month,day,value_ch,prevvalue
1169,LAUMT062094000000003,2018-03-01,2018,M03,15.3,p,03,1,-6.0,21.3
12986,LAUMT483622000000003,2018-03-01,2018,M03,3.1,p,03,1,-3.2,6.3
5069,LAUMT182902000000003,2018-03-01,2018,M03,3.2,p,03,1,-3.0,6.2
1403,LAUMT063370000000003,2018-03-01,2018,M03,7.1,p,03,1,-2.9,10.0
8657,LAUMT352214000000003,2018-03-01,2018,M03,5.5,p,03,1,-2.9,8.4
15053,LAUMT561622000000003,2018-03-01,2018,M03,4.7,p,03,1,-2.9,7.6
2066,LAUMT064970000000003,2018-03-01,2018,M03,8.7,p,03,1,-2.8,11.5
1364,LAUMT063290000000003,2018-03-01,2018,M03,10.2,p,03,1,-2.8,13.0
1247,LAUMT062526000000003,2018-03-01,2018,M03,9.2,p,03,1,-2.8,12.0
1832,LAUMT064210000000003,2018-03-01,2018,M03,6.3,p,03,1,-2.8,9.1


In [7]:
df.sort_values('value').reset_index()['prevvalue'].to_csv('MSANov2015_2.csv')

In [8]:
# Calculate one year change in unemployment rate for each MSA
df.loc[:,'value_ch'] = df['value'].diff(periods=12)
#df = df.dropna()

# Keep only the latest value for each MSA
df = df.reset_index().groupby(['series_id']).tail(1)
df.rename(columns={'series_id':'fullcode'}, inplace=True)

# Take the date for use in chart title
date = df['date'].iloc[0].strftime('%B %Y')

# Combine the two datasets and show 2 rows to verify 
unemp_list = df.merge(MSA, on='fullcode')

# Shorten MSA names for chart labels
unemp_list.loc[:,'MSA'] = unemp_list['area_code'].str.replace('Metropolitan Statistical Area', 'MSA')

In [9]:
# From the change in unemployment rate, mark increases red, decreases green,
#   no change is gray. The size of the circle to draw is the absolute value 
#   of the change times 8. 

for index, row in unemp_list.iterrows():
    if row['value_ch'] > 0:
        unemp_list.loc[index,'color'] = '#FF0000'
        unemp_list.loc[index,'size'] = abs(row['value_ch'])*10+2
        unemp_list.loc[index,'sym'] = '▲{}'.format(unemp_list.loc[index,'value_ch']) # Up Arrow
    if row['value_ch'] < 0:
        unemp_list.loc[index,'color'] = '#32CD32'
        unemp_list.loc[index,'size'] = abs(row['value_ch'])*10+2
        unemp_list.loc[index, 'sym'] = '▼{}'.format(unemp_list.loc[index,'value_ch']) # Down Arrow
    if row['value_ch'] == 0:
        unemp_list.loc[index,'color'] = '#696969'
        unemp_list.loc[index,'size'] = 3   # Just to mark location
        unemp_list.loc[index, 'sym'] = 'unchanged'
    # Create a text column for plotly hover labels
    unemp_list.loc[index,'text'] = '{}<br>{} Rate: {}% ({})'.format(
        unemp_list.loc[index,'MSA'], 
        date,
        unemp_list.loc[index,'value'],
        unemp_list.loc[index,'sym'])

KeyError: 'the label [sym] is not in the [index]'

In [10]:
unemp_list

,index,fullcode,date,year,period,value,footnote_codes,month,day,value_ch,prevvalue,fullname,latitude,longitude,area_code,MSA
0,38,LAUMT011150000000003,2018-03-01,2018,M03,4.5,p,03,1,NaN,6.9,"Anniston, AL",33.695381,-85.839842,"Anniston-Oxford-Jacksonville, AL Metropolitan ...","Anniston-Oxford-Jacksonville, AL MSA"
1,77,LAUMT011222000000003,2018-03-01,2018,M03,3.6,p,03,1,NaN,5.4,"Auburn, AL",32.655091,-85.503643,"Auburn-Opelika, AL Metropolitan Statistical Area","Auburn-Opelika, AL MSA"
2,116,LAUMT011382000000003,2018-03-01,2018,M03,3.5,p,03,1,NaN,5.6,"Birmingham, AL",33.520682,-86.802432,"Birmingham-Hoover, AL Metropolitan Statistical...","Birmingham-Hoover, AL MSA"
3,155,LAUMT011930000000003,2018-03-01,2018,M03,3.5,p,03,1,NaN,5.5,"Daphne, AL",30.571851,-87.855586,"Daphne-Fairhope-Foley, AL Metropolitan Statist...","Daphne-Fairhope-Foley, AL MSA"
4,194,LAUMT011946000000003,2018-03-01,2018,M03,3.7,p,03,1,NaN,6.0,"Decatur, AL",34.624021,-87.057919,"Decatur, AL Metropolitan Statistical Area","Decatur, AL MSA"
5,233,LAUMT012002000000003,2018-03-01,2018,M03,4.0,p,03,1,NaN,6.0,"Dothan, AL",31.223231,-85.390489,"Dothan, AL Metropolitan Statistical Area","Dothan, AL MSA"
6,272,LAUMT012252000000003,2018-03-01,2018,M03,4.2,p,03,1,NaN,6.6,"Florence, AL",34.799810,-87.677251,"Florence-Muscle Shoals, AL Metropolitan Statis...","Florence-Muscle Shoals, AL MSA"
7,311,LAUMT012346000000003,2018-03-01,2018,M03,4.0,p,03,1,NaN,6.1,"Gadsden, AL",34.014380,-86.006668,"Gadsden, AL Metropolitan Statistical Area","Gadsden, AL MSA"
8,350,LAUMT012662000000003,2018-03-01,2018,M03,3.4,p,03,1,NaN,5.3,"Huntsville, AL",34.729847,-86.585901,"Huntsville, AL Metropolitan Statistical Area","Huntsville, AL MSA"
9,389,LAUMT013366000000003,2018-03-01,2018,M03,4.6,p,03,1,NaN,6.6,"Mobile, AL",30.694357,-88.043054,"Mobile, AL Metropolitan Statistical Area","Mobile, AL MSA"


#### Plotly code

In [ ]:
trace1 = {
  "geo": "geo", 
  "hoverinfo": "text", 
  "lat": unemp_list['latitude'], 
  "lon": unemp_list['longitude'], 
  "marker": {
    "color": unemp_list['color'], 
    "line": {
      "color": "#202020", 
      "width": 0.8
    }, 
    "opacity": 0.5, 
    "size": unemp_list['size'], 
    "symbol": "circle"
  }, 
  "mode": "markers", 
  "name": "Unemployment Rate Change by Metro Area", 
  "opacity": 1, 
  "showlegend": True, 
  "text": unemp_list['text'], 
  "type": "scattergeo", 
  "visible": True
}
data = Data([trace1])
layout = {
  "annotations": [
    {
      "x": 0.05, 
      "y": 0.925, 
      "align": "left", 
      "font": {"size": 14}, 
      "showarrow": False, 
      "text": "<b>Change in Unemployment Rate by Metropolitan Statistical Area (MSA)</b><br><i>Change over level in same month of previous year, as of {}</i>".format(date), 
      "xref": "x", 
      "yref": "y"
    }, 
    {
      "x": 0.05, 
      "y": 0.0725, 
      "align": "left", 
      "showarrow": False, 
      "text": "Hover to see details. Green is decrease in unemployment rate, red is increase. Circle size shows amount of change.  <br><b>Source:</b> U.S. Bureau of Labor Statistics, Local Area Unemployment Statistics", 
      "xref": "x", 
      "yref": "y"
    }
  ], 
  "autosize": True, 
  "dragmode": "zoom", 
  "geo": {
    "bgcolor": "#fff", 
    "countrycolor": "#808080", 
    "countrywidth": 0.7, 
    "domain": {
      "x": [0, 1.1], 
      "y": [0, 1.1]
    }, 
    "lataxis": {
      "dtick": 10, 
      "range": [20, 80], 
      "showgrid": False, 
      "tick0": 20
    }, 
    "lonaxis": {
      "dtick": 30, 
      "range": [-180, -50], 
      "showgrid": False, 
      "tick0": -180
    }, 
    "projection": {
      "scale": 1.03, 
      "type": "albers usa"
    }, 
    "resolution": 50, 
    "scope": "usa", 
    "showcountries": False, 
    "showlakes": False, 
    "showland": False, 
    "showrivers": False, 
    "showsubunits": True, 
    "subunitcolor": "#808080", 
    "subunitwidth": 0.7
  }, 
  "hidesources": False, 
  "hovermode": "closest", 
  "paper_bgcolor": "#fff", 
  "separators": ".,", 
  "showlegend": False, 
  "smith": False, 
  "margin": {
    "r": 0, 
    "t": 0, 
    "b": 0, 
    "l": 0
  },
}
fig = Figure(data=data, layout=layout)
plot_url = py.plot(fig, filename='MSA_Map')

#### Generate an html table of results

In [ ]:
# generate html table with data
table = unemp_list[['MSA', 'value', 'value_ch']]
table.columns = ['Metropolitan Statistical Area (MSA) Name', 'Unemp. rate', 'One-year change']
table = table.sort_values('One-year change').drop_duplicates() 
# Sortable css class allows for js script to sort each column
# https://www.kryogenix.org/code/browser/sorttable/
table.to_html('C:/Working/bdecon.github.io/Dash/unemp_table.html', border=0, 
              index=False, classes='sortable')